## PART 2: SQLite

Export the country club data from PHPMyAdmin, and connect to a local SQLite instance from Jupyter notebook 
for the following questions.  


In [1]:
# Import packages
from sqlalchemy import create_engine
import pandas as pd

In [7]:
# Create engine: engine
connection = create_engine('sqlite:///sqlite_db_pythonsqlite.db')

In [8]:
# Execute query and store records in DataFrame: df
df = pd.read_sql_query("Select * from Members", connection)

In [9]:
df.head()

,memid,surname,firstname,address,zipcode,telephone,recommendedby,joindate
0,0,GUEST,GUEST,GUEST,0,(000) 000-0000,,2012-07-01 00:00:00
1,1,Smith,Darren,"8 Bloomsbury Close, Boston",4321,555-555-5555,,2012-07-02 12:02:05
2,2,Smith,Tracy,"8 Bloomsbury Close, New York",4321,555-555-5555,,2012-07-02 12:08:23
3,3,Rownam,Tim,"23 Highway Way, Boston",23423,(844) 693-0723,,2012-07-03 09:32:15
4,4,Joplette,Janice,"20 Crossing Road, New York",234,(833) 942-4710,1,2012-07-03 10:25:05


## Q11: Produce a list of facilities with a total revenue less than 1000.  The output of facility name and total revenue, sorted by revenue.

In [10]:
query = '''SELECT name, revenue FROM (SELECT name, SUM(CASE WHEN memid = 0 THEN guestcost * slots ELSE membercost * slots END) AS revenue
FROM bookings INNER JOIN facilities ON bookings.facid = facilities.facid GROUP BY name) AS inner_table WHERE revenue < 1000 ORDER BY revenue'''
df = pd.read_sql_query(query, connection)
print(df)

            name  revenue
0   Table Tennis      180
1  Snooker Table      240
2     Pool Table      270


## Q12: Produce a report of members and who recommended them in alphabetic surname,firstname order

In [11]:
query = '''SELECT m1.firstname || ' ' || m1.surname AS member, recommended.firstname || ' ' || recommended.surname AS 'recommended by' FROM Members AS m1 INNER JOIN Members recommended On recommended.recommendedby = m1.memid WHERE m1.surname <> "GUEST" AND m1.recommendedby IS NOT NULL ORDER BY m1.surname, m1.firstname'''

In [12]:
df = pd.read_sql_query(query, connection)
print(df)

               member           recommended by
0      Florence Bader         Ramnaresh Sarwin
1       Timothy Baker              Joan Coplin
2      Gerald Butters          Matthew Genting
3      Jemima Farrell            Timothy Baker
4      Jemima Farrell             David Pinker
5     Matthew Genting         Henrietta Rumney
6         David Jones            Douglas Jones
7     Janice Joplette               Nancy Dare
8     Janice Joplette              David Jones
9   Millicent Purview                John Hunt
10         Tim Rownam               Tim Boothe
11       Darren Smith          Janice Joplette
12       Darren Smith           Gerald Butters
13       Darren Smith             Charles Owen
14       Darren Smith               Jack Smith
15       Darren Smith           Anna Mackenzie
16        Tracy Smith  Henry Worthington-Smyth
17        Tracy Smith        Millicent Purview
18        Tracy Smith            Erica Crumpet
19    Ponder Stibbons               Anne Baker
20    Ponder 

## Q13: Find the facilities with their usage by member, but not guests 

In [13]:
query = '''SELECT DISTINCT f.name, m.firstname || ' ' || m.surname AS member FROM Facilities AS f 
INNER JOIN Bookings b ON b.memid = m.memid
INNER JOIN Members m ON f.facid = b.facid
WHERE b.memid <> 0 ORDER BY f.name'''
df = pd.read_sql_query(query, connection)
print(df)

                name             member
0    Badminton Court       Darren Smith
1    Badminton Court     Gerald Butters
2    Badminton Court        Tracy Smith
3    Badminton Court         Nancy Dare
4    Badminton Court         Tim Rownam
..               ...                ...
197   Tennis Court 2         Jack Smith
198   Tennis Court 2  Millicent Purview
199   Tennis Court 2   Henrietta Rumney
200   Tennis Court 2          John Hunt
201   Tennis Court 2      David Farrell

[202 rows x 2 columns]
